In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras import backend as K
from keras.utils import get_custom_objects

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives +
    K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

get_custom_objects().update({'f1_m': f1_m,'precision_m':precision_m, 'recall_m':recall_m})


In [ ]:
# Get the class labels for the training set
from sklearn.svm import SVR
import tensorflow as tf

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import numpy as np


def extract_features(file_path_resnet, file_path_googlenet, file_path_vgg16, train_batches_resnet, train_batches_googlenet, train_batches_vgg16):
    # Load the pre-trained models
    resnet_model = load_model(file_path_resnet)
    googlenet_model = load_model(file_path_googlenet)
    vgg16_model = load_model(file_path_vgg16)

    # Freeze the layers in the models
    for layer in resnet_model.layers:
        layer.trainable = False
    for layer in googlenet_model.layers:
        layer.trainable = False
    for layer in vgg16_model.layers:
        layer.trainable = False

    #resnet_model.summary()
    # Create a new model that outputs the flatten layer# Create a new model that outputs the flatten layer
    resnet_output = resnet_model.layers[-2].output
    resnet_feature_model = Model(inputs=resnet_model.input, outputs=resnet_output)

    googlenet_output = googlenet_model.layers[-2].output
    googlenet_feature_model = Model(inputs=googlenet_model.input, outputs=googlenet_output)

    vgg16_flatten_layer = vgg16_model.layers[-2].output
    vgg16_feature_model = Model(inputs=vgg16_model.input, outputs=vgg16_flatten_layer)

    # Extract features from the models and use PCA to reduce the dimensionality of the features
    #pca = PCA(n_components=174)

    resnet_features = resnet_feature_model.predict(train_batches_resnet)
    #reduced_resnet_features = pca.fit_transform(resnet_features)

    googlenet_features = googlenet_feature_model.predict(train_batches_googlenet)
    #reduced_googlenet_features = pca.fit_transform(googlenet_features)

    vgg16_features = vgg16_feature_model.predict(train_batches_vgg16)
    #reduced_vgg16_features = pca.fit_transform(vgg16_features)

    # Combine the reduced features into a single feature set
    features = np.concatenate((resnet_features, googlenet_features, vgg16_features), axis=1)

    return features


In [ ]:
train_path = '/content/drive/MyDrive/TeaDataSetForML/train'
test_path = '/content/drive/MyDrive/TeaDataSetForML/test'



train_batches_vgg16 = ImageDataGenerator( rescale=1./255,preprocessing_function=tf.keras.applications.vgg16.preprocess_input
).flow_from_directory(train_path, target_size=(224,224), batch_size=10)


test_batches_vgg16 = ImageDataGenerator(rescale=1./255,preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(test_path, target_size=(224,224), batch_size=10,shuffle=False)

train_batches_googlenet = ImageDataGenerator( rescale=1./255,preprocessing_function=tf.keras.applications.vgg16.preprocess_input
).flow_from_directory(train_path, target_size=(224,224), batch_size=10)
test_batches_googlenet = ImageDataGenerator(rescale=1./255,preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(test_path, target_size=(224,224), batch_size=10,shuffle=False)


train_batches_resnet = ImageDataGenerator(rescale=1./25,preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(train_path, target_size=(224,224), batch_size=10)
test_batches_resnet = ImageDataGenerator(rescale=1./25,preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(test_path, target_size=(224,224), batch_size=10,shuffle=False)



Found 706 images belonging to 8 classes.
Found 174 images belonging to 8 classes.
Found 706 images belonging to 8 classes.
Found 174 images belonging to 8 classes.
Found 706 images belonging to 8 classes.
Found 174 images belonging to 8 classes.


In [ ]:
googlenet_path='/content/drive/MyDrive/TeaLeafNet/googlenetmodel9mar2023.h5'
vgg16_path='/content/drive/MyDrive/TeaLeafNet/vgg16TeaLeaf.h5'
resnet50_path='/content/drive/MyDrive/TeaLeafNet/resnet50Mar092023.h5'

In [ ]:
train_features=extract_features(resnet50_path,googlenet_path,vgg16_path,train_batches_resnet,train_batches_googlenet,train_batches_vgg16)

/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


71/71 [==============================] - 338s 5s/step


In [ ]:
train_labels = []
for i in range(train_batches_resnet.__len__()):
    _, labels = train_batches_resnet.next()
    integer_labels = np.argmax(labels, axis=1)
    train_labels.append(integer_labels)
train_labels = np.concatenate(train_labels, axis=0)

In [ ]:
test_features=extract_features(resnet50_path,googlenet_path,vgg16_path,test_batches_resnet,test_batches_googlenet,test_batches_vgg16)

18/18 [==============================] - 84s 4s/step


In [ ]:
test_labels = []
for i in range(test_batches_resnet.__len__()):
    _, labels = test_batches_resnet.next()
    integer_labels = np.argmax(labels, axis=1)
    test_labels.append(integer_labels)
test_labels = np.concatenate(test_labels, axis=0)

In [ ]:

X_train=train_features
print("train_features shape : ",train_features.shape)
X_test=test_features
print("test features shape : ",test_features.shape)

y_train=train_labels
print("len train labels : ",len(train_labels))
y_test=test_labels
print("len test labels : ",len(test_labels))

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)

X_test = scaler.fit_transform(X_test)

print("X_train[0] : ",X_train[0])


train_features shape :  (706, 24)
test features shape :  (174, 24)
len train labels :  706
len test labels :  174
X_train[0] :  [0.00000000e+00 6.12075746e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 9.46567535e-01 1.46871567e-01 8.12573433e-02
 6.47220020e-07 3.35755394e-05 2.41159819e-06 1.30791295e-05
 3.55093135e-07 1.69739479e-07 9.99051154e-01 9.52237286e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 9.99999940e-01]


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=1000)
X_selected = selector.fit_transform(X_train, y_train)
print("X_selected shape : ",X_selected.shape)


KeyboardInterrupt: ignored

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=1000)
Y_selected = selector.fit_transform(X_test, y_test)
print("Y_selected shape : ",Y_selected.shape)


In [ ]:
from sklearn.svm import LinearSVC
# Gaussian kernel
clf_rbf = SVC(kernel='rbf', gamma='auto')
clf_rbf.fit(X_train, y_train)
y_pred_rbf = clf_rbf.predict(X_test)
print(y_pred_rbf)
print(y_test)

print("Accuracy with Gaussian kernel: ", accuracy_score(y_test, y_pred_rbf))


[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 6 6 6 6 6 6 6 6 6 6 6 6 7 6 6 6 6
 6 6 6 6 6 6 6 7 6 6 6 6 7 6 6 6 6 6 6 6 6 7 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 6 6 7 7 6 6 7 6 6 7 6 7 7 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 7 7 6 7 7 7 7 7 7 7 7 7 7 7 7 7 6 7 7 7 7 7 7 7 7 7 7 7 7 7
 6 7 6 7 7 7 7 7 7 6 7 7 7 6 7 7 6 7 7 6 6 6 6 6 6 7]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
Accuracy with Gaussian kernel:  0.10919540229885058


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create a Decision Tree model
dt = DecisionTreeClassifier(max_depth=3)

# Fit the model on the training data
dt.fit(X_train, y_train)
print("y train[0] : ",y_train)

y_pred = dt.predict(X_test)
print("y predication : ",y_pred)
# Evaluate the model on the test data
accuracy = dt.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))


y train[0] :  [1 7 3 7 6 1 1 5 2 2 1 4 6 6 3 7 1 7 7 2 1 7 5 6 1 1 6 6 3 0 6 0 1 2 1 6 3
 2 3 6 1 7 5 5 5 0 3 4 5 4 7 5 2 2 4 6 4 0 7 2 2 2 0 0 4 6 7 2 2 2 2 3 5 3
 1 2 1 0 6 4 5 6 6 5 3 5 7 3 4 7 1 5 4 5 1 2 0 4 7 3 7 3 6 0 5 2 6 6 5 1 1
 1 2 6 7 5 1 0 0 6 4 0 0 4 6 6 0 6 2 5 0 7 1 1 1 1 1 3 2 1 4 6 3 4 4 2 2 2
 1 1 4 3 6 6 0 7 0 4 4 3 3 3 6 7 7 4 4 5 0 2 1 5 4 5 0 6 4 5 6 3 4 5 7 0 7
 6 1 7 4 3 7 2 3 6 6 6 1 7 4 2 7 3 2 5 1 3 7 0 5 4 3 1 7 3 2 6 5 7 7 0 2 4
 3 6 6 0 0 3 6 7 7 0 3 6 6 4 7 1 7 0 0 3 0 3 7 4 3 3 7 2 4 3 6 4 1 0 6 4 1
 2 7 1 7 5 7 5 6 4 5 0 3 6 7 1 1 3 1 4 6 6 6 6 2 4 0 3 0 0 0 0 3 2 1 7 2 2
 5 7 7 0 7 4 0 6 4 2 5 0 7 2 4 4 1 1 2 0 6 6 4 4 4 4 1 2 6 3 2 2 5 4 3 7 0
 3 0 3 2 5 5 6 0 5 1 4 3 6 0 1 1 3 1 6 2 4 7 7 6 6 1 6 0 2 4 3 2 1 1 2 1 6
 7 5 1 5 0 4 7 0 7 7 0 4 6 1 0 7 2 0 1 6 6 4 7 1 0 1 3 4 2 6 6 5 0 2 7 7 2
 6 2 6 3 6 1 7 0 7 7 3 4 3 4 3 6 6 0 3 6 7 6 6 5 5 6 6 3 2 5 0 7 1 2 7 7 4
 1 1 2 2 0 4 7 3 2 0 7 1 3 1 7 7 0 7 0 7 6 2 2 7 2 7 6 7 4 0 6 3 1 3 1 2 3
 1 3 4 6 1 

In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:

# Train a Random Forest classifier with 100 trees
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = clf.predict(X_test)
print(y_pred)
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

[1 1 1 1 1 4 0 1 1 1 0 1 4 1 1 1 4 1 1 1 6 2 2 6 2 3 3 2 6 0 3 3 3 6 3 4 3
 7 3 3 6 5 6 4 1 4 4 6 4 1 6 0 4 4 4 1 4 6 1 1 6 4 1 0 2 2 3 4 7 0 3 2 0 2
 1 7 2 0 6 3 2 6 2 2 4 1 2 1 1 5 4 4 1 7 0 1 1 4 1 7 4 1 1 1 6 6 3 3 6 6 6
 6 6 6 6 3 6 6 7 7 2 7 7 3 7 2 6 7 1 7 7 4 7 6 2 2 7 7 7 3 1 7 7 6 4 7 1 7
 3 1 7 7 6 7 7 2 6 4 7 2 3 1 7 7 7 7 2 6 2 3 4 3 1 6]
Accuracy: 0.13218390804597702


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


# Create a Gaussian Naive Bayes classifier and fit it to the training data
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Use the trained classifier to make predictions on the testing data
y_pred = gnb.predict(X_test)

# Evaluate the accuracy of the classifier
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.22413793103448276


In [ ]:
import numpy as np
!pip install scikit-fuzzy
!pip install --upgrade scikit-fuzzy
import skfuzzy as fuzz
from fuzz import classifier, cluster
import numpy as np

#Create FMM classifier
fmm = classifier.FuzzyClassifier(c=2.0, m=2.0)

# Train the classifier on the training data
fmm.fit(X_train, y_train)


# Make predictions on the testing data
y_pred = fmm.predict(X_test)

# Print the predicted classes
print(y_pred)






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ModuleNotFoundError: ignored

In [ ]:
from keras.models import Model, load_model
from keras.layers import concatenate, Average, Dense, Input, multiply
from keras.optimizers import Adam
import numpy as np
import tensorflow as tf
from keras.utils import plot_model
from keras.layers import Flatten, Reshape
from keras.utils import Sequence


# Load the pre-trained models
googlenet = load_model(googlenet_path)
vgg16 = load_model(vgg16_path)
resnet50 = load_model(resnet50_path)

# Rename the common layers in each model
for layer in googlenet.layers:
    layer._name = 'googlenet_' + layer.name
for layer in vgg16.layers:
    layer._name = 'vgg16_' + layer.name
for layer in resnet50.layers:
    layer._name = 'resnet50_' + layer.name

# Freeze the layers
for layer in googlenet.layers[:-1]:
    layer.trainable = False
for layer in vgg16.layers[:-1]:
    layer.trainable = False
for layer in resnet50.layers[:-1]:
    layer.trainable = False

# Create a new model that combines the outputs of the pre-trained models

googlenet_output = Flatten()(googlenet.get_layer('googlenet_avg_pool_5_3x3/1').output)
print("googlenet_output.shape",googlenet_output.shape)
vgg16_output = Flatten()(vgg16.get_layer('vgg16_block5_pool').output)
print("vgg16_output.shape",vgg16_output.shape)

resnet50_output = Flatten()(resnet50.get_layer('resnet50_average_pooling2d_6').output)
print("resnet50_output.shape",resnet50_output.shape)


merged_output = concatenate([googlenet_output, vgg16_output, resnet50_output])
print("merged_output",merged_output)
dense1= Dense(256, activation='softmax')(merged_output)
output_tensor = Dense(8, activation='softmax')(dense1)

print("googlenet input ",googlenet.input)
print("vgg16 input ",vgg16.input)
print("resnet50 input ",resnet50.input)


hybrid_model = Model(inputs=[googlenet.input, vgg16.input, resnet50.input], outputs=output_tensor)


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


googlenet_output.shape (None, 1024)
vgg16_output.shape (None, 25088)
resnet50_output.shape (None, 32768)
merged_output KerasTensor(type_spec=TensorSpec(shape=(None, 58880), dtype=tf.float32, name=None), name='concatenate/concat:0', description="created by layer 'concatenate'")
googlenet input  KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'googlenet_input_1'")
vgg16 input  KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
resnet50 input  KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_7'), name='input_7', description="created by layer 'resnet50_input_7'")


In [ ]:
train_path = '/content/drive/MyDrive/TeaDataSet/train'
valid_path = '/content/drive/MyDrive/TeaDataSet/valid'
test_path = '/content/drive/MyDrive/TeaDataSet/test'


train_batches_vgg16 = ImageDataGenerator( rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest',preprocessing_function=tf.keras.applications.vgg16.preprocess_input
).flow_from_directory(train_path, target_size=(224,224), batch_size=10,shuffle=False)
valid_batches_vgg16 = ImageDataGenerator(rescale=1./255,preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(valid_path, target_size=(224,224), batch_size=10,shuffle=False)
test_batches_vgg16 = ImageDataGenerator(rescale=1./255,preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(test_path, target_size=(224,224), batch_size=10,shuffle=False)


Found 706 images belonging to 8 classes.
Found 87 images belonging to 8 classes.
Found 87 images belonging to 8 classes.


In [ ]:
train_batches_googlenet = ImageDataGenerator( rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest',preprocessing_function=tf.keras.applications.vgg16.preprocess_input
).flow_from_directory(train_path, target_size=(224,224), batch_size=10,shuffle=False)
valid_batches_googlenet = ImageDataGenerator(rescale=1./255,preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(valid_path, target_size=(224,224), batch_size=10,shuffle=False)
test_batches_googlenet = ImageDataGenerator(rescale=1./255,preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(test_path, target_size=(224,224), batch_size=10,shuffle=False)


Found 706 images belonging to 8 classes.
Found 87 images belonging to 8 classes.
Found 87 images belonging to 8 classes.


In [ ]:
train_batches_resnet = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(train_path, target_size=(224,224), batch_size=10,shuffle=False)
valid_batches_resnet = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(valid_path, target_size=(224,224), batch_size=10,shuffle=False)
test_batches_resnet = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(test_path, target_size=(224,224), batch_size=10,shuffle=False)


Found 706 images belonging to 8 classes.
Found 87 images belonging to 8 classes.
Found 87 images belonging to 8 classes.


In [ ]:
def custom_generator(generator):
    while True:
        """batch_images, batch_labels = next(generator)
        batch_resnet_input = resnet_generator.preprocessing_function(batch_images.copy())
        batch_vgg_input = vgg_generator.preprocessing_function(batch_images.copy())
        batch_googlenet_input = googlenet_generator.preprocessing_function(batch_images.copy())"""

        #batch_images, batch_labels = next(generator)
        resnet_batch_images,resnet_batch_labels = next(train_batches_resnet)
        googlenet_batch_images,googlenet_batch_labels=next(train_batches_googlenet)
        vgg16_batch_images,vgg16_batch_labels=next(train_batches_vgg16)
        yield [googlenet_batch_images, vgg16_batch_images, resnet_batch_images], googlenet_batch_labels


In [ ]:
def custom_generatorValid(generator):
    while True:

        resnet_batch_images,resnet_batch_labels = next(valid_batches_resnet)
        googlenet_batch_images,googlenet_batch_labels=next(valid_batches_googlenet)
        vgg16_batch_images,vgg16_batch_labels=next(valid_batches_vgg16)
        yield [googlenet_batch_images, vgg16_batch_images, resnet_batch_images], googlenet_batch_labels


In [ ]:
resnet_generator=ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

vgg_generator=ImageDataGenerator( rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest',preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)


googlenet_generator= ImageDataGenerator( rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest',preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

NameError: ignored

In [ ]:
train_batches_custom_generator  = ImageDataGenerator( ).flow_from_directory(train_path, target_size=(224,224), batch_size=10)
valid_batches_custom_generator  = ImageDataGenerator( ).flow_from_directory(valid_path, target_size=(224,224), batch_size=10)


Found 706 images belonging to 8 classes.
Found 87 images belonging to 8 classes.


In [ ]:
# Train the new model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.initializers import glorot_uniform

epochs = 100
initial_lrate = 0.001
def decay(epoch, steps=100):
  initial_lrate = 0.01
  drop = 0.96
  epochs_drop = 8
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate
sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)


hybrid_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

callback_list=[EarlyStopping(monitor="val_loss",patience=250),ModelCheckpoint(filepath="/content/drive/MyDrive/TeaLeafNetV2.0sagarlogin.h5",monitor="val_loss",save_best_only=True,verbose=1)]

hybrid_model.fit_generator(custom_generator(train_batches_googlenet), epochs=1000, validation_data=custom_generatorValid(valid_batches_googlenet),verbose=1,callbacks=callback_list)


/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
<ipython-input-15-fb014b195d5a>:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hybrid_model.fit_generator(custom_generator(train_batches_googlenet), epochs=1000, validation_data=custom_generatorValid(valid_batches_googlenet),verbose=1,callbacks=callback_list)


Epoch 1/1000
   1608/Unknown - 16391s 10s/step - loss: 2.1066 - accuracy: 0.0375